In [188]:
# function definitions are in the script, no definitions in this ipynb allowed!
include("PES_models_bonding_features.jl") # need to reload everytime a change occurs

multirestart (generic function with 1 method)

In [2]:
# test using actual data:
H_data = readdlm("data/h3/h3_data.txt")
# load atomic coordinates:
H_coord = npzread("data/h3/h3_coord.npy")
R = H_data[:,1:end-1]; V = H_data[:, end]
n_data = size(R)[1]

6032

In [3]:
idxes = shuffleobs(1:n_data); # shuffle indexes

In [4]:
# split:
id_train, id_test = splitobs(idxes, at=0.8)
R_train = R[id_train,:]; V_train = V[id_train];
R_test = R[id_test,:]; V_test = V[id_test]
H_coord_train = H_coord[id_train,:,:]; H_coord_test = H_coord[id_test,:,:];

In [119]:
const_r_xy = 1.4172946 # H2
#const_r_xy = 1.9369 #OH+
N = 2 # hyperparam
ρ = f_ρ(R, const_r_xy)
q = f_q_bump(N, ρ)
i = f_i(q) # should be transformed to unique integers
ϵ = f_ϵ(i, q)
# for h:
α = f_α(ϵ)
β = f_β(ϵ)
#ub = 1.; lb = -1; θ = rand(2*N+2) .* (ub-lb) .+ lb; # +2 since it starts from 0th index
θ = ones(2*N+2);

In [120]:
n_data, n_d = size(R)
u = similar(R)
w = similar(u)
h = zeros(n_data, n_d, N+1) #stores the first sub primitive
compute_u_w_h!(u, w, h, θ, q, α, β, i, N)
#@benchmark compute_u_w_h!($u, $w, $h, $θ, $q, $α, $β, $i, $N)

In [121]:
x = similar(h)
y = similar(x)
compute_hw_gw!(x, y, w, h, q, N)
#@benchmark compute_hw_gw!($x, $y, $w, $h, $q, $N)

In [122]:
ρ,u,w,x,y = BUMP_feature(θ, R, const_r_xy, N);
@benchmark BUMP_feature($θ, $R, $const_r_xy, $N)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  174.500 μs …   6.800 ms  ┊ GC (min … max):  0.00% … 94.51%
 Time  (median):     224.900 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   292.651 μs ± 336.137 μs  ┊ GC (mean ± σ):  19.31% ± 15.24%

  ▄█▆▄▂                                                    ▁▁   ▁
  ██████▇▅▄▃▄▅▅█▇▆▇▆▄▄▆▅▃▅▄▄▃▃▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆███▇ █
  174 μs        Histogram: log(frequency) by time       2.02 ms <

 Memory estimate: 2.35 MiB, allocs estimate: 22.

In [189]:
display(x)
display(y)
n_data, n_d, _ = size(x)
b = Array{Float64}(undef, n_data, n_d, 5)
concat_BUMP(b, x, y, 5)

6032×3×3 Array{Float64, 3}:
[:, :, 1] =
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          6.92183e-6
 0.0       0.000175034  0.000251095
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          1.90742e-6
 0.0       0.0          1.40248e-6
 0.0       0.0          0.000133393
 ⋮                      
 0.258754  0.651427     0.733013
 0.387945  0.573971     0.694344
 0.60972   0.656929     0.658869
 0.61047   0.648742     0.649685
 0.0       0.832285     0.835488
 0.0       0.845661     0.857254
 0.0       0.833046     0.83883
 0.0       0.861325     0.863779
 0.0       0.84103      0.84326
 0.0       0.847137     0.8693
 0.0       0.8467       0.858908
 0.0       0.839594     0.843443

[:, :, 2] =
 0.986362  0.999394  0.999796
 0.986362  0.999998  0.999999
 0.995901  0.995901  0.998824
 0.995901  0.995901  0.999706
 0.995901

6032×3×3 Array{Float64, 3}:
[:, :, 1] =
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          6.85556e-6
 0.0       0.000170072  0.000243053
 0.0       0.0          0.0
 0.0       0.0          0.0
 0.0       0.0          1.89555e-6
 0.0       0.0          1.39461e-6
 0.0       0.0          0.000129943
 ⋮                      
 0.162344  0.275382     0.276667
 0.215947  0.265657     0.277352
 0.271059  0.275782     0.275913
 0.271157  0.275172     0.275247
 0.0       0.26089      0.259944
 0.0       0.256704     0.252545
 0.0       0.260668     0.25892
 0.0       0.250957     0.249966
 0.0       0.258224     0.257502
 0.0       0.256203     0.247639
 0.0       0.256352     0.251909
 0.0       0.25868      0.257442

[:, :, 2] =
  0.124327    0.0431398    0.0298546
  0.124327    0.00613871   0.0053048
  0.0826823   0.0826823    0.0540395
  0.0826823   0

In [190]:
@benchmark concat_BUMP($x, $y, 5)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  86.800 μs … 150.000 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     87.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   87.534 μs ±   1.569 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▅██▁                   ▂▄ ▄▃▂▁                              ▂
  ▃████▅▁▅▄▃▃▁▃▁▁▃▃▄▄▄▁▃▆▇██▁████▇▆▁▇████▇▁▇▆▆▇▇▁▇▆▆▅▆▅▁▆▆▅▄▄▄ █
  86.8 μs       Histogram: log(frequency) by time      91.9 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [96]:
idxer = atom_indexer(3)
U = f_U_bas_BUMP(idxer, u, w, ρ, e_pow)
@benchmark f_U_bas_BUMP($idxer, $u, $w, $ρ, $e_pow)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   93.600 μs …   5.665 ms  ┊ GC (min … max):  0.00% … 98.16%
 Time  (median):     106.100 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   125.538 μs ± 172.849 μs  ┊ GC (mean ± σ):  12.63% ±  9.05%

  ▁▆█▇▅▃▁                        ▁                              ▂
  ███████▇▆▆▆▅▆▃▄▅▆▇▅▄▁▃▃▁▁▁▁▃▄▆▇██▇▆▅▁▄▄▁▄▅▄▅▃▃▁▃▃▁▄▃▃▃▃▃▁▁▁▃▄ █
  93.6 μs       Histogram: log(frequency) by time        361 μs <

 Memory estimate: 708.47 KiB, allocs estimate: 50.

In [177]:
A = ones(Int(1e5),4,3); B = zeros(Int(1e5),4,3) .+ 2
function alloctest(A, B)
    i = 1; acount = bcount = 1
    C = zeros(Int(1e5),4,6)
    for i=1:6
        if i%2 == 0
            C[:, :, i] = @view B[ :, :, bcount]
            bcount += 1
        else
            C[:, :, i] = @view A[ :, :, acount]
            acount += 1
        end
        i += 1
    end
    return C
end
alloctest(A, B)

100000×4×6 Array{Float64, 3}:
[:, :, 1] =
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 ⋮              
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0

[:, :, 2] =
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 ⋮              
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  

In [176]:
@benchmark alloctest($A, $B)

BenchmarkTools.Trial: 817 samples with 1 evaluation.
 Range (min … max):  3.950 ms … 21.214 ms  ┊ GC (min … max):  0.00% … 43.41%
 Time  (median):     5.192 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   6.120 ms ±  2.945 ms  ┊ GC (mean ± σ):  16.79% ± 20.56%

  ▃▃   ▂█▅▂                                             ▂▁    
  █████████▇▆▇▅▅▄▄▄▁▁▄▁▁▄▄▁▁▁▁▄▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▄▄▁▅▁▆▅▁▅▄██▆▇ ▇
  3.95 ms      Histogram: log(frequency) by time     14.6 ms <

 Memory estimate: 15.26 MiB, allocs estimate: 2.

In [178]:
a = b = 0.
a += 1
b

0.0